In [ ]:
## Bibliotecas
import pandas as pd
import time
import random
import numpy as np
from web3 import Web3
import web3
import mysql.connector

#conectar ao banco de dados
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="123456",
    database="sys"
)

pd.set_option('display.max_colwidth', None)

cursor = db.cursor()

#premiacao
def prize(UserWinner):
    #premiacao
    for winner in UserWinner: #percorre os vencedores
        while True:
            addressWinner = input(f"{winner[0]} de CPF: {winner[1]} informe o endereço da sua carteira do MetaMask  (exemplo de formato: 0xe34FC27644eaE4271435F66784F11Cf31e797E1a): ") #input dos endereco da MetaMask
            try:
                address2 = Web3.to_checksum_address(addressWinner) #checa o endereco informado
                break
            except(ValueError): #garente que o endereco informado esta certo
                print("Informe a hex string da sua carteira (exemplo de formato: 0xe34FC27644eaE4271435F66784F11Cf31e797E1a): ")
        
        w3 = Web3(Web3.HTTPProvider('')) #se conecta a uma servidor
       

        from_account = '' #enderaco de onde vai sair os eth

        private_key = '' #chave privada da API


        address1 = Web3.to_checksum_address(from_account) #checa o endereco

        nonce = w3.eth.get_transaction_count(address1) #nonce da transicao
        # informacao que irao para a api
        tx = { 
            'nonce': nonce, 
            'to': address2,
            'value': w3.to_wei(0.000001, 'ether'), #preco e moeda de transicao 
            'gas': 21000, #valor de gas
            'gasPrice': w3.to_wei(40, 'gwei')

        }
        #enviar para api
        signed_tx = w3.eth.account.sign_transaction(tx, private_key)
        try:
            tx_Transaction =  w3.eth.send_raw_transaction(signed_tx.rawTransaction)
            print("\nPREMIACAO ENVIADA!!!!!\n")
        except(ValueError): #tratamento de excecao para se nao tiver valor sufuciente de ETH na conta
            print("\nInfelizmente o preco do eth esta muito caro para ficar fazendo tantas transações\n")
            
            

#sorteio dos numeros extras

def draw (numsDraw):
    
    numWin = 0 #variael para saber se há vencedores
    cont = 0 #variael para percorrer os numeros sorteados
    i = 0
    cursor = db.cursor()
    cursor.execute("SELECT B.id, A.nameUser, A.cpf, B.n1,B.N2, B.N3, B.N4, B.N5 FROM BET B INNER JOIN USERDATA A  ON B.cpf = A.cpf")#Seleciona do banco o nome, cpf e os numeros apostados para ver se existe algum ganhador
    resultBet = cursor.fetchall()
    while True:
      if cont < 25: #if para controlar o numero maximo de apostas adicionais   
        
        for row in resultBet:#for que percorre todas as apostas retornadas do banco
                arrayBet = [row[3], row[4],row[5],row[6],row[7]] #salva em um array os numeros apostados
                if set(arrayBet).issubset(set(numsDraw)):#compara os numeros apostados com os numeros sorteados
                  numWin +=1 
                  print("TEMOS UM GANHADOR!!!!!!!!!!!!!!!!!!!!!!\n")
                  time.sleep(1.5)
                  print(f"Se apresente {row[1]} que tem o cpf {row[2]} para receber seu premio!!!!\n")
                  print("--------------------\n")
                  cursor.execute(f'UPDATE userdata SET winner = 1 WHERE cpf = {row[2]} ;') #altera na tabela userdata, a variavel Winner para 1, onde o cpf seja igual dos numeros apostados
                  time.sleep(1.5)  
                  db.commit()

        if numWin == 0: #se ainda nao tem nenhum vencedor, sorteio outro numero
                  print('''AH NAO!
Parece que nao temos nenhum ganhador ainda
Vamos sortear mais um numero!\n''')
                  time.sleep(1.5)
                  while True:
                      

                      numsDraw[5+cont] = str(random.randrange(1, 51)) #sorteio um novo numero aleatorio entre 1 e 50
                      
                      if len(numsDraw) == len(set(numsDraw)):# garante que o novo numero sorteado nao e repedito
                        print(f"O novo numero sortado e...\n {numsDraw[5+cont]}!!!")
                        time.sleep(2.0)  
                        print("Vamos ver se temos um ganhador\n")
                        print("--------------------\n")
                        time.sleep(1.5)
                        break
                          
                      
                          
                       



        else: #se ja tem algum vencerdor e retornado os numeros sorteados
            return numsDraw;

            break

                
                
      else: #se os 25 sorteio extras forem realizados e nao ter nenhum vencedor e retornado os numeros sorteados
        time.sleep(1.0)
        print('''NUMERO MAXIMO DE NUMEROS SORTEADOS
INFELIZMENTE NAO TEMOS NENHUM VENCEDOR DESSE SORTEIO
BOA SORTE NA PROXIMA\n''')
        print("--------------------\n")
        time.sleep(1.0)
        return numsDraw;
      cont += 1
       
#APURAÇÃO
       
def counting(numsDraw):
        time.sleep(1.0)
        print('''Sorteio ja realizado
Vamos para a apuracao das apostas
Espere um pouquinho enquando realizamos os calculos''')
        time.sleep(2.0)
        print("Os numeros sorteados foram: ")
        dfDraw = pd.DataFrame(numsDraw, columns = ['Numeros sorteados']) #cria um dataframe, com a variael numsDraw, para deixar os numeros sorteados mais faceis de vizualizar
        display(dfDraw) #mostra o dataframe
        time.sleep(2.0)
        print(f"A quantidade de sorteios extras realizados: {len(numsDraw)-5}\n")#diminui 5 do tamanho total de numsDraw para saber quantos sorteio extras foram realizados
        time.sleep(2.0)
        cursor.execute("SELECT  nameUser, cpf FROM userdata WHERE winner = 1 ORDER BY nameUser")#seleciona o nome, em ordem alfabetica, e o cpf, dos usuario vencedores
        UserWinner = cursor.fetchall()
        time.sleep(2.0)
        print(f"Quantidade de apostas vencedoras: {len(UserWinner)}") #mostra a quantidade de vencedores
        time.sleep(2.0)

        print("\nOs vencedores sao: ")        
        dfWinner = pd.DataFrame(UserWinner, columns = ['Vencerdores', 'CPF']) #mostra o nome e cpf dos vencedores num dataframe
        if len(UserWinner) == 0: #se nao tem nenhum vencedor exibe uma mensagem informando
            print("Nao ouve vencedores\n")
        else: display(dfWinner) #monstra o dataframe dos ganhadores

        cursor.execute("SELECT  n1, n2, n3, n4, n5 FROM bet") #seleciona todos os numeros apostados
        allNum = cursor.fetchall()
        time.sleep(2.0)

        list1 = []
        save = list()
        for num in allNum: #percorre todos os numeros informados e coloca eles em um mesmo array
            list1 = list1+list(num)
        list1 = [eval(m) for m in list1]

        for l in range (51) : # percorre os numeros de 1 a 50
            times = list1.count(l+1) #conta quantas vezes os numeros de 1 a 50 aparecem no array
            if list1.count(l+1) > 0: #se for maior que 0
                save.append([(times, l+1)])#salva numa lista de array a quantidade de vezes e o numero
        time.sleep(2.0)
        print("Nro apostados \t Qtd de apostas")
        for i in sorted(save, reverse =True): #percorre todos os array da lista ordenando eles por ondem que mais aparece
            print(f"{i[0][1]} \t\t {i[0][0]}")
        time.sleep(2.0)

        
        
        
        
        
        
#main        

DrawDone = False #variavel para saber se o sorteio ja foi realizado
while True:

    print("=================== SISTEMA DE CONTROLE DE APOSTAS DE ETHEREUM  ===================")
    if DrawDone: print("Status Sorteio: Realizado") #mostra o status do sorteio
    else: print("Status Sorteio: Em andamento")
    time.sleep(1.0)
    #menu inicial de funcionalidades
    print('''
    Bem vindo!

    Escolha umas das opcoes abaixo:
    [1] Novas edicao do sorteio
    [2] Realizar nova aposta
    [3] Exibir a lista de aposta
    [4] Finalizar apostas e executar o sorteio
    
    ''')
    try:
        func = int(input("Opcao Escolhida: ")) #seleciona a funcionalidade
    except:
        print("Valor nao numerico informado") #verifica se esta em formato numerico
        func = -1
    print('\n=====================================================================================\n')
    if func == 1: #funcionalidade 1
        while True:
            try:
                #pergunta se quer reiniciar todos os dados e comecar o sorteio 
                confirm = int(input('''Tem certeza que desejas apagar todos os dados do sorteio e comecar uma novo?
[1]Nao
[2]Sim

'''))
                if confirm == 1: #se nao, volta para o menu inicial
                    break
                if confirm == 2:# se sim
                    DrawDone = False #define a variavel false, para um novo sorteio
                    cursor.execute("TRUNCATE TABLE bet;") #apaga todos os valores da tabela bet
                    cursor.execute("TRUNCATE TABLE userdata;") #apaga todos os valores da tabela userdata
                    cursor.execute("ALTER TABLE BET AUTO_INCREMENT = 1000;") #reinicia a contagem do ID
                    db.commit()
                    break
                #garante as excecoes
                if confirm > 2 and confirm < 1:
                    print("Informe ou a opcao 1 ou a opcao 2")
            except: 
                print("Valor numerico nao informado")
      
            
            
    if func == 2: #funcionalidade 2
     cpfDuplicate = 0 #variavel para saber se o cpf e duplicado

     if DrawDone: #se o sorteio ja foi realizado , mostra uma mensagem e volta para o menu inicial
         print('''Sorteio ja realizado, nao aceitando novas apostas
Para conferir a apuracao selecione a opcao 4
Para realizar um novo sorteio selecione a opcao 1''')
     else:
         nameUser = input("Informe o nome do usuario: ")#salva o nome do usuario
         nameUser = "'"+nameUser+"'" #deixa o nome em formato para ser salvo no banco de dados

         while True:#laco de repeticao ate um cpf valido
          #VARIAVEIS CALCULO CPF
           j = 10
           k = 11
           sumJ = 0
           sumK = 0
           accept = 0
           try:
               if cpfDuplicate == 2:#se o cpf foi duplicado e deseja usar ele para outra aposta, sai do laco de repeticao
                    break;
               cpf = input("Informa o CPF (somente o numero): ") #salva o cpf
               if cpf.isdigit() and len(cpf) == 11: #confira se o cpf é composto so por numeros e tem 11 de tamanho
                for i in range(9): #verifica o primeiro numero verificador
                  sumJ +=  int(cpf[i])*j #soma os 9 primeiros numeros do cpf * a variavel j
                  j -=1
                for i in range(10):  #verifica o segundo numero verificador
                  sumK +=  int(cpf[i])*k #soma os 10 primeiros numeros do cpf * a variavel j
                  k -=1

                remainderJ = sumJ%11 #resto da soma dos numeros dividido por 11
                remainderK = sumK%11 #resto da soma dos numeros dividido por 11

                if remainderJ == 0 or remainderJ == 1: #se o resto for 1 ou 0
                    if int(cpf[9]) == 0: #o 10 numero tem q ser 0
                        accept += 1 #variael para saber se o numero verificador esta certo
                elif int(cpf[9]) == 11 - remainderJ:#se o resto nao for 1 ou 0, o numero verificador tem que ser 11 - o resto
                      accept += 1 #variael para saber se o numero verificador esta certo
                if remainderK == 0 or remainderK == 1: #se o resto for 1 ou 0
                    if int(cpf[10]) == 0: #o 11 numero tem q ser 0
                        accept += 1  #variael para saber se o numero verificador esta certo
                elif int(cpf[10]) == 11 - remainderK: #se o resto nao for 1 ou 0, o numero verificador tem que ser 11 - o resto
                      accept += 1#variael para saber se o numero verificador esta certo
                        
                if accept == 2: #se a variavel for 2 entao o cpf e valido
                  print("CPF VALIDO")
                  cursor = db.cursor()
                  comandUser = f'INSERT INTO USERDATA(nameUser, cpf, winner) VALUES({nameUser}, {cpf}, 0)' #salva o cpf e o nome no banco de dados
                  cursor.execute(comandUser)
                  break;
                else: print("CPF INVALIDO")
               else: print("CPF INVALIDO")
           except: #se o cpf já esta no banco de dados
            while True:
                try:
                    #pergunta se quer usar um novo cpf ou usar o mesmo cpf para outra aposta
                 cpfDuplicate = int(input('''CPF ja cadastrado
[1] Cadastrar novo cpf
[2] Usar o mesmo cadastro para outra aposta
         ''')) 
                 if cpfDuplicate == 2:#usar o mesmo cpf
                    break;
                 if cpfDuplicate == 1:#cadastrar um novo cpf
                    break;
                 if cpfDuplicate > 2 or cpfDuplicate < 1:
                    print("Informe 1 ou 2")
                except: 
                    print("Valor numerico nao informado")
         numsBet = [-1,-2,-3,-4,-5] #array onde serao salvos os numeros apostados

         for i in range(5): #for que percorre os 5 numeros
           while True: #while para garantir que nenhum numero foi informado de forma errada
            try:
                numsBet[i] = int(input(f"Informe o {i+1}° numero da aposta (1 a 50) \nDigite 0 para escolher um numero surpresinha(aleatorio): "))
                if numsBet[i] == 0: #se for 0, um numero aleatorio é sorteado
                    while True:
                        numsBet[i] = random.randrange(1, 51) #sorteia um numero aleatorio
                        if len(numsBet) != len(set(numsBet)): #se o numero sorteado ja foi escolhido, outro numero e sorteado
                          numsBet[i] = random.randint(1, 50)
                        else: break;
                if numsBet[i] < 0 or numsBet[i] > 50: #se o numero escolhido pelo usuario for maior que 0 ou menor, a funcao repete
                    print("Numero invalido, informe um numero entre 1 e 50")
                elif len(numsBet) != len(set(numsBet)):#se um numero repetido foi escolhido, a funcao repete
                    print("Informe um numero diferentes dos escolhidos anteriormente")
                else:
                    print(f"O {i+1}° Numero apostado foi: {numsBet[i]}\n") # mostra o numero escolhido
                    break;
            except:
              print("Opção escolhida em formato nao numerico") #garante que foi salva em int

         comandBet = f'INSERT INTO BET(cpf, n1, n2, n3, n4, n5) VALUES({cpf},{numsBet[0]},{numsBet[1]}, {numsBet[2]}, {numsBet[3]}, {numsBet[4]})' #salva os numeros no banco de dados, usando o cpf para saber de quem foi a aposta

         cursor.execute(comandBet)
         db.commit()
         time.sleep(1.5)
         print('''
Aposta realiza com sucesso!
Boa Sorte!
''')

    if func == 3: #funcionalidade 3
        time.sleep(1.0)
        cursor = db.cursor()
        cursor.execute("SELECT B.id, A.nameUser, A.cpf, B.n1,B.N2, B.N3, B.N4, B.N5 FROM BET B INNER JOIN USERDATA A  ON B.cpf = A.cpf") #busca o id, nome, cpf e os numeros apostados, usando o innver join para juntar pelo cpf as tabelas
        result = cursor.fetchall()
        
        if len(result) != 0: #se tiver alguma aposta realizada
            dfResult = pd.DataFrame(result, columns = ['ID','Name','CPF','N1','N2','N3','N4','N5']) #cria um data frame com os dados
            display(dfResult) #exibe o dataframe
            time.sleep(1.5)
            time.sleep(1.5)
        else: 
            print("Nenhuma aposta registrada") # se nenhuma aposta e realizada exibe a mensagem
            time.sleep(1.5)


    if func == 4: #funcionalidade 4
      cursor = db.cursor()
      cursor.execute("SELECT * FROM bet") #seleciona as apostas
      betIsempty = cursor.fetchall()
      if DrawDone: #se o sorteio ja foi realizado, so mostra a apuracao
        counting(numsDraw) #chama def da apuracao
      elif len(betIsempty) != 0:
          
          if len(betIsempty) == 0: #confere se existe alguma aposta realizada
              print("Nenhuma aposta realizada")
              break; 
          #array de valores negativos onde serao salvo os numeros sorteados
          numsDraw = [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29,-30]
          while True: #while para ter uma entrada de dados valida
                try:
                  confirm = int(input('''
Desejas nao fazer mais nenhuma aposta e realizar o sorterio?
[1] NAO
[2] SIM      ''')) #confirma que nenhuma aposta mais sera realizada e comeca o sorteio

                  if confirm == 2:#
                      break;
                  if confirm == 1:
                      break;
                  if confirm > 2 or confirm < 1:
                      print("Informe 1 ou 2")
                except:
                      print("Valor numerico não informado")
        
          if confirm == 2: #se confirm e igual a 2 comeca a ser sorteado
            print('''
APOSTAS ENCERRADAS
VAMOS PARA O SORTEIO
''')
            time.sleep(1.0)

            for i in range(5): #percorre os 5 primeiros espaco do array
                print(f'''
O {i+1}° NUMERO SORTEADO É...
''')
                time.sleep(1.0)

                while True:
                  numsDraw[i] = str(random.randrange(1, 51)) #gera um numero aleatorio
                  if len(numsDraw) != len(set(numsDraw)): #verifica se o numero aleatorio nao e repetido
                      numsDraw[i] = str(random.randint(1, 51)) #gera um novo numero aleatorio se for repetido
                  else: break;
                print(f"{numsDraw[i]}!!!!!!!!!!!!!")
                time.sleep(2.0)

            print("Agora vamos ver se temos um ganhador!\n")
            time.sleep(1.0)
            

            numsDraw  = draw(numsDraw) #chama a funcao draw() para ver se tem o vencedor e se precisar realizar novas rodadas do sorteio
            
            DrawDone = True #define sorteio como realizado
            
            numsDraw = np.array([int(x) for x in numsDraw]) #deixa os numeros como int
            numsDraw = numsDraw[numsDraw > 0] #deixa somente os numeros positivos no array
            
            counting(numsDraw) #chama a funcao para realizar a apuracao
            
            cursor.execute("SELECT  nameUser, cpf FROM userdata WHERE winner = 1 ORDER BY nameUser")#seleciona os nomes e cpf dos ganhadores
            UserWinner = cursor.fetchall()
            print("Agora vamos para a distribuição do premio: ")
            if len(UserWinner) != 0:
                prize(UserWinner) #funcao para distribuir premio
            else: print("Infelizmente nao tivemos vencedores")
      else:print("Nenhuma Aposta registrada")
    


    if func > 4 or func < 1: #garante que o numero informado e menor que 4 e maior que 1
        print("Por favor, insira um numero de 1 a 4\n")
        time.sleep(2)

=================== SISTEMA DE CONTROLE DE APOSTAS DE ETHEREUM  ===================
Status Sorteio: Em andamento

    Bem vindo!

    Escolha umas das opcoes abaixo:
    [1] Novas edicao do sorteio
    [2] Realizar nova aposta
    [3] Exibir a lista de aposta
    [4] Finalizar apostas e executar o sorteio
    
    
Opcao Escolhida: 1


Tem certeza que desejas apagar todos os dados do sorteio e comecar uma novo?
[1]Nao
[2]Sim

2
=================== SISTEMA DE CONTROLE DE APOSTAS DE ETHEREUM  ===================
Status Sorteio: Em andamento

    Bem vindo!

    Escolha umas das opcoes abaixo:
    [1] Novas edicao do sorteio
    [2] Realizar nova aposta
    [3] Exibir a lista de aposta
    [4] Finalizar apostas e executar o sorteio
    
    
Opcao Escolhida: 3


Nenhuma aposta registrada
=================== SISTEMA DE CONTROLE DE APOSTAS DE ETHEREUM  ===================
Status Sorteio: Em andamento

    Bem vindo!

    Escolha umas das opcoes abaixo:
    [1] Novas edicao do sorteio
    [2